In [5]:
import os
import s3fs
import pandas as pd

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "atoubert-ensae"
FILE_KEY_S3 = "X_train_Hi5.csv"
FILE_PATH_S3 = BUCKET + "/Hackathon Hiparis/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    x_train = pd.read_csv(file_in, sep=",")

/tmp/ipykernel_30368/3036990934.py:13: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  x_train = pd.read_csv(file_in, sep=",")


In [7]:
x_train.describe

<bound method NDFrame.describe of          row_index piezo_station_department_code  \
0                0                            01   
1                1                            01   
2                2                            01   
3                3                            01   
4                4                            01   
...            ...                           ...   
2830311    3294080                            95   
2830312    3294081                            95   
2830313    3294082                            95   
2830314    3294083                            95   
2830315    3294084                            95   

             piezo_station_update_date  piezo_station_investigation_depth  \
0        Sun Jul 14 13:00:02 CEST 2024                              20.00   
1        Sun Jul 14 13:00:02 CEST 2024                              35.60   
2        Sun Jul 14 13:00:02 CEST 2024                              35.22   
3        Sun Jul 14 13:00:02 CEST

In [9]:
train_sample = x_train.sample(n=10_000, random_state=42)

In [11]:
train_sample.head()

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
2613083,3067028,33,Fri Jun 28 07:31:38 CEST 2024,123.0,Gironde,33517,LE PORT (SOUSSANS-33),['334AG04'],3.59,07795X0106/PZEM9,...,NaN,NaN,NaN,NaN,19,1692.0,23100,6.3,18.5,Average
1268326,1334103,49,Fri Jan 12 10:28:05 CET 2024,93.5,Maine-et-Loire,49194,FONTAINE-MILONJurassique,['139AP14'],40.00,04552X0111/PZ2,...,-0.281423,47.471283,49194.0,18620902.0,8.8,5815.0,23560,8.8,12.5,Average
234393,246594,27,Mon Sep 23 19:38:51 CEST 2024,12.0,Eure,27391,PUITS DE L'ANCIENNE GARE (MARCILLY-SUR-EURE),['121AS01'],75.00,01807X0051/S1,...,1.365070,48.713810,28404.0,29117529.0,1.3,1588.0,25340,16.6,15.9,High
1670802,1756732,44,Fri Jun 28 07:31:38 CEST 2024,8.0,Loire-Atlantique,44025,Forage de LA RIVIERE,['104AB01'],5.00,04503X0068/PZ2,...,-1.881925,47.275110,44045.0,8572341.0,1,4005.0,23620,18.6,19.8,Average
2804886,3267483,77,Thu Sep 19 14:26:45 CEST 2024,15.0,Seine-et-Marne,77341,PIÉZOMÈTRE DE NOYEN-SUR-SEINE (NOYEN-SUR-SEINE...,['928AC01'],57.50,02606X0112/G5,...,NaN,NaN,NaN,NaN,0,384.0,24270,0,8.3,Very Low


In [12]:
train_sample.tail(10)

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
1356563,1426779,40,Fri Jun 28 07:31:38 CEST 2024,72.00,Landes,40158,F1 Lapeyrade,['320AA01'],144.00,09264X0038/F,...,-0.098517,43.875774,32127.0,1.985332e+07,5.2,278.0,18590,65.2,0.9,Very High
112182,117997,24,Fri Jun 28 07:31:38 CEST 2024,10.00,Dordogne,24080,PECESTIER (CAPDROT - 24),['346AA03'],232.00,08317X0024/PUITS2,...,0.878274,44.665158,24195.0,1.156138e+07,2.5,461.0,21100,1,0,Very High
1998954,2100777,54,Fri Jun 28 07:31:38 CEST 2024,119.30,Meurthe-et-Moselle,54371,S15 - Pz M2 Paradis aval - Calcaires du Dogger...,['139AP99'],215.82,01377X0212/M02,...,5.942921,49.199276,54371.0,2.021503e+07,4.3,1077.0,24430,9.7,18.3,Low
167004,175686,21,Fri Jun 28 07:31:38 CEST 2024,50.00,Côte-d'Or,21584,forage BRGM (Saulieu-21),['208AA01'],567.00,04973X0005/PZ1,...,4.228165,47.232075,21560.0,1.936866e+06,1.1,2324.0,20760,23.1,5.6,Very High
2737977,3197519,17,Fri Jun 28 07:31:38 CEST 2024,40.00,Charente-Maritime,17385,L'AUBIER F,['348AE23'],7.10,06576X0030/F,...,NaN,NaN,NaN,NaN,9.4,6627.0,23040,7.5,5.9,Average
28346,29791,36,Fri Jun 28 07:31:38 CEST 2024,138.86,Indre,36110,VAVRE - AEP VAVRE N 1 - PARCELLE 1369 - STAT...,"['141AC01', '141AG05', '143AB99', '143AD99']",184.00,05933X0006/FAEP3,...,1.581543,46.502656,36012.0,2.787306e+08,25,259.0,20790,31.2,0,Average
1678943,1765242,67,Fri Jun 28 07:31:38 CEST 2024,9.00,Bas-Rhin,67482,PIEZOMETRE,NaN,140.43,BSS004EPHV/X,...,7.782502,48.532152,67482.0,8.268934e+07,0.1,291313.0,20260,4.6,2.1,Low
1784633,1876137,38,Mon Oct 04 22:03:42 CEST 2021,6.00,Isère,38298,PUITS - LES OVES SUD (LE PEAGE DE ROUSSILLON -...,['712AA01'],137.80,BSS003ZFSK/X,...,4.753204,45.403553,38425.0,3.240429e+09,0,6508.0,18930,5.1,10.5,Average
813048,855057,72,Fri Jun 28 07:31:38 CEST 2024,101.00,Sarthe,72182,BOIS DE FAY,['123AC05'],85.00,03936X0056/PZ,...,0.178346,47.640774,72176.0,2.211338e+07,4.1,1546.0,21750,20.7,2.8,High
1625530,1709203,50,Tue Sep 24 13:16:39 CEST 2024,12.39,Manche,50271,PUITS DE LINGEARD (LINGEARD) - 50,['171AA03'],217.00,02101X0044/P,...,-1.032545,48.941748,50592.0,7.277293e+06,0,87.0,N/A - résultat non disponible,0,0,Low


In [15]:
nan_columns = train_sample.columns[train_sample.isna().all()]
print("Colonnes contenant uniquement des NaN :", nan_columns.tolist())

Colonnes contenant uniquement des NaN : ['meteo_radiation_IR']
